In [ ]:

#     В ЭТОМ КОДЕ:

#     + кластеризуем посты в группе ССММ (про выборы/нет, анонсы/результаты выборов)
#     + посты с анонсами выборов конвертируем в словарики (имя, курс-поток-группа, текст программы)
#     + посты с результатами выборов конвертируем в словарики со всей информацией об итогах выборов
#     + последние два пункта адаптировать для постов про магистров
#   !!! расставляем флаги предпочтений кандидатов, исходя из текста программ


In [2]:
import requests
from bs4 import Tag, NavigableString, BeautifulSoup
import re
from datetime import datetime, date, time

import pandas as pd


In [ ]:
'''
ЧТО ПОФИКСИТЬ:
- БАГ: путает блок текста в шапке репоста с блоком текста в самом репосте (пример: https://vk.com/wall-33327655_2261)
- находить посты с перечнем избранных должностей
- https://vk.com/wall-33327655_5491 - запись удалена, но код этого не видит
- https://vk.com/wall-33327655_5492 - запись не найдена, а код сломался
- https://vk.com/wall-33327655_9414 - код сломался
- были парные кандидаты, найти и расщепить
'''

# СДЕЛАТЬ ПАРСИНГ ОБСУЖДЕНИЙ В ГРУППЕ:
#     треды с кандидатами 1-3 созывов
#     треды с отчетами о собраний 1-2(3?) созывов


In [134]:
# ссылка на репост без шапки, репостнутая запись - без текста (обновление аватарки):

req_announcement = requests.get('https://vk.com/wall-33327655_9846')
html_doc = req_announcement.text
print(html_doc)

# типы постов:

#     https://vk.com/wall-33327655_2250 -- пост с тегом
#     https://vk.com/wall-33327655_2261 -- пост без тега
#     https://vk.com/wall-33327655_2255 -- post not found
#     https://vk.com/wall-33327655_2262 -- коммент под постом с тегом
#     https://vk.com/wall-33327655_3330 -- репост без шапки поста с адресом 104154624_1
#     https://vk.com/wall-33327655_8489 -- репост с шапкой поста по адресу 77014399_6331


            <!DOCTYPE html>
      <html lang="ka" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="МФК в осеннем семестре    #образование_ссмм    Запись на Межфакультетские курсы в осеннем семестре 2021/2022 учебного года откроется 3 окт" />
        <meta property="og:description" content="МФК в осеннем семестре    #образование_ссмм    Запись на Межфакультетские курсы в осеннем семестре 2021/2022 учебного года откроется 3 окт" />
        <link re

## CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK

In [137]:
'''
CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK
'''

def vk_post_info_by_link(link_vk, language):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
#     дефолтные значения
    content_type = 'no text'
    content_subtype = 'other'
    parent_link = ''
    text_lines = []
    link_year_no = 0
    link_month_no = 0
    link_day_no = 0
    link_date = date(1970, 1, 1)
    link_date_time = ''
    link_time = ''
    months_dict = {
        'russian': ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'],
        'english': ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
#         https://en.wikipedia.org/wiki/Georgian_calendar
        'georgian': ['იან', 'თებ', 'მარ', 'აპრ', 'მაი', 'ივნ', 'ივლ', 'აგვ', 'სექ', 'ოქტ', 'ნოე', 'დეკ']
    }
    months = months_dict[language]

    # ПРОВЕРЯЮ, СУЩЕСТВУЕТ ЛИ ПОСТ
    #     <div class="service_msg service_msg_error">Запись не найдена.</div>
    flg_service_msg_error = soup.find('div', attrs={'class': 'service_msg service_msg_error'})
    if flg_service_msg_error:
        link_type = 'page not found'
    else:
        link_type = 'post'
        # когда ссылка на пост: <link rel="canonical" href="https://vk.com/wall-33327655_2225" />
        # когда ссылка на коммент: <link rel="canonical" href="https://vk.com/wall-33327655_8793?reply=8800" />
        
        # ПРОВЕРЯЮ, НЕ КОММЕНТ ЛИ ЭТО
        rel_canonical = soup.find('link', attrs={'rel': 'canonical'})
        flg_rel_canonical_reply = re.findall('reply=', str(rel_canonical))
#             + ДОБАВИТЬ ПОЛЕ МАТЕРИНСКОЙ ССЫЛКИ
        if flg_rel_canonical_reply:
            link_type = 'comment'
            
        # ПРОВЕРЯЮ, НЕ РЕПОСТ ЛИ ЭТО
        flg_repost = soup.find('div', attrs={'data-copy': True})
        if flg_repost:
            link_type = 'repost'
#             <div class="wall_item single_wall_item bl_item mark_top_verified" data-copy='-77014399_6331'>
            parent_link = 'https://vk.com/wall' + flg_repost['data-copy']
    
    if link_type in ['post','repost']:
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ ПОСТА ПОСТРОЧНО    
        # <div class="wi_body">
        #     <div class="pi_text">

        wi_body = soup.find('div', attrs={'class': 'wi_body'})
        if wi_body is None:
            announcement_list = []
            result_list = []
        else:
            pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
            if pi_text is None:
                announcement_list = []
                result_list = []
            else:
                text_lines = [line for line in pi_text.stripped_strings]
                    # определить тип контента
                    # анонсы: 'Орфография и пунктуация сохранены.'
                    # результаты: 'Таким образом, выборы признаются состоявшимися'
                announcement_list = ['announcement' for line in pi_text.stripped_strings if line.find('Орфография и пунктуация сохранены.') >= 0]
                result_list = ['result' for line in pi_text.stripped_strings if line.find('Таким образом, выборы признаются состоявшимися.') >= 0]
                content_type = 'text'
                if any(election_tag in text_lines for election_tag in ['#выборы@stud_mm','#выборы_ссмм']):
                    content_type = 'election'
    
                if len(announcement_list) > 0:
                    content_subtype = 'announcement'
                elif len(result_list) > 0:
                    content_subtype = 'result'
        
        # ВЫТАСКИВАЮ ДАТУ
        # <div class="wi_info">
        #     <span class="wi_date">
        wi_info = soup.find('div', attrs={'class': 'wi_info'})
        if wi_info:
            wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
            date_list = [date for date in wi_date.stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
            link_date_time = date_list[0]
            link_time = re.findall(r'[\d]+\:[\d]+', date_list[0])[0]
        
    if link_type == 'comment':
        reply_link = link_vk[link_vk.index('_')+1:]
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ КОММЕНТА ПОСТРОЧНО    
#             <div class="ReplyItem__body">    180% голосов. Это выборы на мехмате? Это проценты чего?</div>
#             <div class="ReplyItem__date">
#             <a class="item_date" href="https://m.vk.com/wall-33327655_2254?reply=2262#reply2262" rel="noopener">22 &#4316;&#4317;&#4308;&#4315; 2014</a></div>
#         reply_body = soup.find('div', attrs={'class': 'ReplyItem__body'})
        reply_bodies = soup.find_all('div', attrs={'class': 'ReplyItem__body'})
        reply_dates = soup.find_all('a', attrs={'class': 'item_date'})
        reply_links = [ff['href'][ff['href'].index('#reply')+6:] for ff in soup.find_all('a', attrs={'class': 'item_date'})]
#         reply_no = reply_links.index(reply_link)
#         if reply_body is None:
#             announcement_list = []
#             result_list = []
#         else:
        if reply_link in reply_links:
            reply_no = reply_links.index(reply_link)
            content_type = 'text'
            text_lines = [line for line in reply_bodies[reply_no].stripped_strings]
            date_list = [date for date in reply_dates[reply_no].stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
        else:
            content_subtype = 'deleted (?)'
            
        parent_link = rel_canonical['href'][: rel_canonical['href'].index('?')]
    
    dict_link = {}
    dict_link[link_vk] = {
        'link_type': link_type, 
        'content_type': content_type,
        'content_subtype': content_subtype,
        'parent_link': parent_link,
        
        'link_date': link_date, 
        'link_year_no': link_year_no, 
        'link_month_no': link_month_no, 
        'link_day_no': link_day_no, 
        'link_date_time': link_date_time, 
        'link_time': link_time, 
        
        'list_of_lines': text_lines
    }
    
    return dict_link


In [138]:
# тест
# vk_post_info_by_link('https://vk.com/wall-33327655_5106', 'georgian') # -- пост с анонсом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2262', 'georgian') # -- коммент под постом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2263', 'georgian') # -- коммент под постом +
# vk_post_info_by_link('https://vk.com/wall-33327655_3330', 'georgian') # -- репост без шапки +
# vk_post_info_by_link('https://vk.com/wall-33327655_2261', 'georgian') # -- репост без шапки - ПОЧЕМУ ТУТ text, а не no text
# vk_post_info_by_link('https://vk.com/wall-33327655_8489', 'georgian') # -- репост с шапкой +
vk_post_info_by_link('https://vk.com/wall-33327655_9846', 'georgian') # -- удаленный коммент (?)


{'https://vk.com/wall-33327655_9846': {'link_type': 'comment',
  'content_type': 'no text',
  'content_subtype': 'deleted (?)',
  'parent_link': 'https://vk.com/wall-33327655_9819',
  'link_date': datetime.date(1970, 1, 1),
  'link_year_no': 0,
  'link_month_no': 0,
  'link_day_no': 0,
  'link_date_time': '',
  'link_time': '',
  'list_of_lines': []}}

## functions for processing of posts about elections

### functions for posts with announcements

In [32]:

# вытаскивает из строки номера курса и потока (в предположении, что они в этой строке есть)

# для обычных потоков
def year_stream_no_by_line(line):
    if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line):
        year_stream_line = (re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line))[0]
        year_stream_line_d = re.sub('\D', '', year_stream_line)
        year_no = int(year_stream_line_d[0])
        stream_no = int(year_stream_line_d[1])
    elif re.findall(r'\D(\d\.\d)\D', line):
        year_stream_line = (re.findall(r'\D(\d\.\d)\D', line))[0]
#         print(year_stream_line)
#         print(year_stream_line[0])
        year_no = year_stream_line[0]
        stream_no = year_stream_line[2]
    
    year_stream_no = str(year_no) + '.' + str(stream_no)
    return year_stream_no, year_no, stream_no

# для магистратуры
def magister_year_by_line(line):
#     если номер года записан цифрами
    if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line))[0]
#         print(year_stream_line)
        year_no = int(year_stream_line[1])
        year_stream_no = 'm' + str(year_no)
#     если номер года записан буквами
    elif re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line))[0]
#         print(year_stream_line)
        year_no = 0
        if str.lower(year_stream_line[1]) == 'перв':
            year_no = 1
        elif str.lower(year_stream_line[1]) == 'втор':
            year_no = 2
        year_stream_no = 'm' + str(year_no)
    return year_stream_no, year_no

# примеры для магистратуры:
# МАГИСТРЫ ПЕРВОГО ГОДА ОБУЧЕНИЯ (выборы завтра на второй паре в аудитории 12-03) +
# МАГИСТРЫ, 2 ГОД ОБУЧЕНИЯ +
# МАГИСТРЫ 2 ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ 1-ГО ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ, 1 КУРС +

# line = 'МАГИСТРЫ, 1 КУРС'
# magister_year_by_line(line)

In [5]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО КАНДИДАТОВ (по списку строк с текстом)


# по списку строк с текстом составляем СЛОВАРЬ: имя: курс-поток, группа, номер первой строки в тексте
def stream_nomination_by_list(list_of_lines):
    nominated_dict = {}
#     year_stream_no, year_no, stream_no = '', 0, 0
    
    for i in range(len(list_of_lines)):
        
#         if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
#             year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
            
#         В НАЧАЛЕ ШАБЛОНА БЫЛО \s (убрал, так как иногда забывают пробел, а алгоритм всё равно жадный)
        if len(re.findall(r'[\w\s]+,\sгруппа\s[\d]{3}', list_of_lines[i])) > 0:
            name_line_no = i
            name_line = (re.findall(r'[\w\s]+,\sгруппа\s[\d]{3}', list_of_lines[i]))[0]
            name_list = re.split(', ',name_line)
            name = (re.sub('\W\s', '', name_list[0])).strip().title()
            academic_group_no = int(re.sub('\D', '', name_list[1]))
            another_one_dict = {}
            another_one_dict[name] = {
#                 'year_stream_no': year_stream_no, 
#                 'year_no': year_no, 
#                 'stream_no': stream_no, 
                'academic_group_no': academic_group_no, 
                'name_line_no': name_line_no
            }
            nominated_dict = {**nominated_dict, **another_one_dict}
            
    # первый и последний индексы программы кандидата в списке
    name_list = list(nominated_dict.keys())
    start_index_list = [nominated_dict[key]['name_line_no'] for key in list(nominated_dict.keys())]
    last_index_list = [i-1 for i in start_index_list if start_index_list.index(i) > 0]
    last_index_list.append(len(list_of_lines)-1)
    
    dict_of_nominated_upd = {}
    
    for name in name_list:
        dict_of_nominated_upd[name] = [start_index_list[name_list.index(name)], last_index_list[name_list.index(name)]]
    
    # обновить словарь: имя: номер группы, текст программы
    for name in dict_of_nominated_upd:
        program_lines = []

        for k in range(dict_of_nominated_upd[name][0]+1, dict_of_nominated_upd[name][1]+1):
            program_lines.append(list_of_lines[k])
        program_txt = '\n'.join(program_lines)

        nominated_dict[name].update({'program_txt': program_txt})
        del nominated_dict[name]['name_line_no']

    return nominated_dict


# вся инфа про номинации на потоке
def stream_nomination_info_by_dict(list_of_lines):
    year_stream_no = ''
    year_no, stream_no = 0, 0
    
    for i in range(len(list_of_lines)):
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
                
    dict_nominations = stream_nomination_by_list(list_of_lines)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'dict_nominations': dict_nominations
    }
    
    return dict_stream_elections



### functions for posts with results

In [28]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО РЕЗУЛЬТАТЫ ВЫБОРОВ (по списку строк с текстом)


# по списку строк с кандидатами составляем СЛОВАРЬ: имя: голоса, статус
def stream_election_result_by_list(nominated_with_votes):
    dict_nominated = {}

    for line in nominated_with_votes:
        name = ((re.findall(r'[\w\s]+', line))[0]).strip().title()
        votes_line = (re.findall(r'\s[\d]+\sголос', line))[0]
        votes_cnt = int(re.sub('\D', '', votes_line))
        dict_nominated[name] = {'votes_cnt': votes_cnt}
        if (re.findall(r' избран', line)) and not (re.findall(r' не избран', line)):
            elected_flg = 1
        else:
            elected_flg = 0
        dict_nominated[name].update({'elected_flg': elected_flg})
    return dict_nominated


# вся инфа про результаты выборов на потоке
def stream_election_info_by_dict(list_of_lines):
    nominated_with_votes = []
    year_stream_no = ''
    year_no, stream_no = 0, 0
    voted_cnt, stream_size, invalid_cnt, NOTA_cnt = 0, 0, 0, 0
    
    for i in range(len(list_of_lines)):
                    
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]) or re.findall(r'\D(\d\.\d)\D', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
            
        if re.findall('Явка: ', list_of_lines[i]):
            turnout_line = (re.findall(r'[\d]+\s[\w]+\sиз\s[\d]+', list_of_lines[i]))[0]
            voted_cnt = int(((re.findall(r'[\d]+\s', turnout_line))[0]).strip())
            stream_size = int(((re.findall(r'\s[\d]+', turnout_line))[0]).strip())
            
        if len(re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i])) == 0: # отдельно, т.к. может такой записи не быть
            pass
        else:
            invalid_line_no = i
            invalid_line = (re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i]))[0]
            invalid_cnt = re.sub('\D', '', invalid_line)
            
        if re.findall('Против всех', list_of_lines[i]):
            NOTA_line_no = i
            NOTA_line = (re.findall(r'\s[\d]+\sголос', list_of_lines[i]))[0]
            NOTA_cnt = re.sub('\D', '', NOTA_line)
            
        if re.findall('[\w]+\:\s[\d]+\sголос', list_of_lines[i]) and not (re.findall('Против всех', list_of_lines[i])):
            nominated_with_votes.append(list_of_lines[i])
            
        if re.findall('Подтверждено ответственн', list_of_lines[i]):
            organizer_line = (re.findall(r'за\sвыборы\s[\w\s,]+', list_of_lines[i]))[0]
            organizer_line = organizer_line[10::]
            organizer_list = re.split(', | и ',organizer_line)
    
    dict_nominated_with_votes = stream_election_result_by_list(nominated_with_votes)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'stream_size': stream_size, 
        'voted_cnt': voted_cnt,
        'invalid_cnt': invalid_cnt,
        'NOTA_cnt': NOTA_cnt,
        'organizers': organizer_list,
        'dict_nominated_with_votes': dict_nominated_with_votes
    }
    
    return dict_stream_elections


## Processing of all election posts for 4-11 convocations

### launching

In [159]:
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С АНОНСАМИ ВЫБОРОВ
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С РЕЗУЛЬТАТАМИ ВЫБОРОВ

from itertools import chain

announcement_links = {}
result_links = {}

for num in chain(
#     range(10054, 10070) #, # test
    range(2200, 2360) #, # созыв 4 +
#     range(3140, 3530) #, # созыв 5 +
#     range(5040, 5360) #, # созыв 6 +
#     range(6540, 6720) #, # созыв 7 +
#     range(7470, 7780), # созыв 8 +
#     range(8240, 8420), # созыв 9 +
#     range(9100, 9400), # созыв 10 +
#     range(9800, 10040) # созыв 11 +
):
    link = 'https://vk.com/wall-33327655_' + str(num)

    test_dict = vk_post_info_by_link(link, 'georgian')
    print('{0} : {1} \t\t/ {2} \t\t/ {3}'.format(link, test_dict[link]['link_type'], test_dict[link]['content_type'], test_dict[link]['content_subtype']))

    for d in test_dict.values():
        if d['content_type'] == 'election' and d['content_subtype'] == 'announcement':
            announcement_links = {**announcement_links, **test_dict}
        if d['content_type'] == 'election' and d['content_subtype'] == 'result':
            result_links = {**result_links, **test_dict}


for link in announcement_links:
    announcement_links[link].update({'nominated_info': stream_nomination_info_by_dict(announcement_links[link]['list_of_lines'])})
    del announcement_links[link]['list_of_lines']
    
for link in result_links:
    result_links[link].update({'stream_election_info': stream_election_info_by_dict(result_links[link]['list_of_lines'])})
    del result_links[link]['list_of_lines']

# print(announcement_links)
# print(result_links)



https://vk.com/wall-33327655_2200 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2201 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2202 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2203 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2204 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2205 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2206 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2207 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2208 : post 		/ election 		/ other
https://vk.com/wall-33327655_2209 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2210 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2211 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2212 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2213 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_

https://vk.com/wall-33327655_2324 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2325 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2326 : post 		/ election 		/ result
https://vk.com/wall-33327655_2327 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2328 : post 		/ election 		/ result
https://vk.com/wall-33327655_2329 : post 		/ election 		/ result
https://vk.com/wall-33327655_2330 : post 		/ election 		/ result
https://vk.com/wall-33327655_2331 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2332 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2333 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2334 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2335 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2336 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2337 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2338 : comment 		/ text 		/ other
https://vk.com/wa

In [140]:
# поправки из-за единичных нестандартных случаев

if 'https://vk.com/wall-33327655_7695' in result_links.keys():
    result_links['https://vk.com/wall-33327655_7695']['stream_election_info']['year_stream_no'] = 'm1'
    result_links['https://vk.com/wall-33327655_7695']['stream_election_info']['year_no'] = 1
if 'https://vk.com/wall-33327655_9205' in result_links.keys():
    result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['year_stream_no'] = '6.3'
    result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['year_no'] = 6
    result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['stream_no'] = 3
if 'https://vk.com/wall-33327655_9267' in result_links.keys():
    result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['year_stream_no'] = '6.1'
    result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['year_no'] = 6
    result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['stream_no'] = 1

# result_links['https://vk.com/wall-33327655_9267']


### creating tables about candidates and streams

In [141]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

announcement_streams = {}
for link in announcement_links.keys():
    announcement_streams[link] = {
        'year_x_year_stream': str(announcement_links[link]['link_year_no']) + '_' + announcement_links[link]['nominated_info']['year_stream_no'],
        'link_date': announcement_links[link]['link_date'],
        'link_year_no': announcement_links[link]['link_year_no'], 
        'link_month_no': announcement_links[link]['link_month_no'], 
        'link_day_no': announcement_links[link]['link_day_no'], 
        'link_time': announcement_links[link]['link_time'], 
        'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
        'year_no': announcement_links[link]['nominated_info']['year_no'], 
        'stream_no': announcement_links[link]['nominated_info']['stream_no']
    }
# announcement_streams

announcement_streams_df =  pd.DataFrame(announcement_streams.items(), columns=['link', 'params'])
announcement_streams_df_2 = pd.concat([announcement_streams_df.drop(['params'], axis=1), announcement_streams_df['params'].apply(pd.Series)], axis=1)
announcement_streams_df_3 = announcement_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
announcement_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no
0,https://vk.com/wall-33327655_9970,2021_1.1,2021-10-15,2021,10,15,11:11,1.1,1,1
1,https://vk.com/wall-33327655_9950,2021_1.2,2021-10-14,2021,10,14,10:20,1.2,1,2
2,https://vk.com/wall-33327655_9969,2021_1.3,2021-10-15,2021,10,15,10:44,1.3,1,3
3,https://vk.com/wall-33327655_9936,2021_2.1,2021-10-13,2021,10,13,11:44,2.1,2,1
4,https://vk.com/wall-33327655_9901,2021_2.2,2021-10-12,2021,10,12,10:40,2.2,2,2
5,https://vk.com/wall-33327655_9932,2021_2.3,2021-10-13,2021,10,13,11:21,2.3,2,3
6,https://vk.com/wall-33327655_9900,2021_2.5,2021-10-12,2021,10,12,10:10,2.5,2,5
7,https://vk.com/wall-33327655_9894,2021_3.1,2021-10-11,2021,10,11,11:48,3.1,3,1
8,https://vk.com/wall-33327655_9889,2021_3.2,2021-10-08,2021,10,8,11:44,3.2,3,2
9,https://vk.com/wall-33327655_9895,2021_3.3,2021-10-11,2021,10,11,11:58,3.3,3,3


In [ ]:
# !!!! РАЗОБРАТЬСЯ С ПОВТОРНЫМИ ПОСТАМИ ВНУТРИ ОДНОГО ПОТОКА


In [142]:
announcement_streams_df_3.to_csv('ssmm_announcement_info_conv11.csv', sep='\t', encoding='utf-8', index=False)

In [143]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

announcement_names = {}
for link in announcement_links.keys():
    for name in announcement_links[link]['nominated_info']['dict_nominations'].keys():
        announcement_names[name] = {
            'link_year_no_x_name': str(announcement_links[link]['link_year_no']) + "_" + "_".join(name.split()),
            'link_year_no': announcement_links[link]['link_year_no'], 
            'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
            'year_no': announcement_links[link]['nominated_info']['year_no'], 
            'stream_no': announcement_links[link]['nominated_info']['stream_no'], 
            'academic_group_no': announcement_links[link]['nominated_info']['dict_nominations'][name]['academic_group_no'], 
            'program_txt': announcement_links[link]['nominated_info']['dict_nominations'][name]['program_txt'],
            'link': link
        }
# announcement_names

announcement_names_df =  pd.DataFrame(announcement_names.items(), columns=['name', 'params'])
announcement_names_df_2 = pd.concat([announcement_names_df.drop(['params'], axis=1), announcement_names_df['params'].apply(pd.Series)], axis=1)
announcement_names_df_2.head(10)


,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link
0,Малиновский Георгий Андреевич,2021_Малиновский_Георгий_Андреевич,2021,5.4,5,4,531,Всем привет!​\nЯ избираюсь в Студенческий сове...,https://vk.com/wall-33327655_9853
1,Мельникова Дарья Владимировна,2021_Мельникова_Дарья_Владимировна,2021,5.4,5,4,531,Всем привет!\nВ Студенческий совет избираюсь в...,https://vk.com/wall-33327655_9853
2,Третьяков Петр Максимович,2021_Третьяков_Петр_Максимович,2021,5.3,5,3,523,"I, Petr Tretyakov, stand here today, humbled b...",https://vk.com/wall-33327655_9858
3,Клюев Андрей Игоревич,2021_Клюев_Андрей_Игоревич,2021,5.3,5,3,523,Избираюсь в Студсовет четвёртый раз. До этого ...,https://vk.com/wall-33327655_9858
4,Евдокимова Софья Александровна,2021_Евдокимова_Софья_Александровна,2021,5.3,5,3,523,"В студсовет иду в третий раз, хотелось бы прод...",https://vk.com/wall-33327655_9858
5,Пчельникова Елена Павловна,2021_Пчельникова_Елена_Павловна,2021,5.2,5,2,507,"В любом деле, как мне кажется, даже минимально...",https://vk.com/wall-33327655_9860
6,Шереметьев Владислав Валерьевич,2021_Шереметьев_Владислав_Валерьевич,2021,4.3,4,3,425,Всем салют!\nКто я?\nЗаконопослушный парень с ...,https://vk.com/wall-33327655_9882
7,Манахова Анастасия Николаевна,2021_Манахова_Анастасия_Николаевна,2021,4.3,4,3,423,Всем привет! Я - Настя)\nВ прошлом созыве из-з...,https://vk.com/wall-33327655_9882
8,Водолеев Марк Дмитриевич,2021_Водолеев_Марк_Дмитриевич,2021,3.5,3,5,341,"Меня зовут Марк Водолеев, и я уже больше года ...",https://vk.com/wall-33327655_9887
9,Лоншаков Никита Артемович,2021_Лоншаков_Никита_Артемович,2021,3.5,3,5,341,Привет! В предыдущем учебного году мне выдалас...,https://vk.com/wall-33327655_9887


In [ ]:
# присвоить ssmm_id в порядке публикации постов (дата, время)
# ИЛИ другим способом ОТСОРТИРОВАТЬ КАНДИДАТОВ (сейчас они в порядке появления в постах в порядке выхода постов, вроде норм)

In [144]:
announcement_names_df_2.to_csv('ssmm_candidates_info_conv11.csv', sep='\t', encoding='utf-8', index=False)

In [151]:
# full table for 4-11 convocations:

list_of_df = []
for i in range(4,11+1):
    df = pd.read_csv('ssmm_announcement_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
    list_of_df.append(df)

ssmm_announcement_info = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
ssmm_announcement_info


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no
0,https://vk.com/wall-33327655_2220,2014_1.1,2014-11-18,2014,11,18,23:10,1.1,1,1
1,https://vk.com/wall-33327655_2225,2014_1.2,2014-11-19,2014,11,19,21:07,1.2,1,2
2,https://vk.com/wall-33327655_2234,2014_1.3,2014-11-21,2014,11,21,1:42,1.3,1,3
3,https://vk.com/wall-33327655_2227,2014_2.1,2014-11-19,2014,11,19,21:12,2.1,2,1
4,https://vk.com/wall-33327655_2314,2014_2.1,2014-11-26,2014,11,26,0:00,2.1,2,1
...,...,...,...,...,...,...,...,...,...,...
153,https://vk.com/wall-33327655_9858,2021_5.3,2021-10-05,2021,10,5,10:35,5.3,5,3
154,https://vk.com/wall-33327655_9853,2021_5.4,2021-10-04,2021,10,4,10:47,5.4,5,4
155,https://vk.com/wall-33327655_9978,2021_6.1,2021-10-16,2021,10,16,10:25,6.1,6,1
156,https://vk.com/wall-33327655_9982,2021_6.3,2021-10-16,2021,10,16,10:50,6.3,6,3


In [152]:
ssmm_announcement_info.to_csv('ssmm_announcement_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)


In [160]:

list_of_df = []
for i in range(4,11+1):
    df = pd.read_csv('ssmm_candidates_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
    list_of_df.append(df)

ssmm_candidates_info = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
ssmm_candidates_info


,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link
0,Суркова Анастасия Юрьевна,2014_Суркова_Анастасия_Юрьевна,2014,2.2,2,2,209,"«Подавая заявку в прошлом году в студсовет, яв...",https://vk.com/wall-33327655_2202
1,Денисов Илья Олегович,2014_Денисов_Илья_Олегович,2014,2.2,2,2,207,"«Вечер добрый, коллеги!\nЯ бы хотел подать зая...",https://vk.com/wall-33327655_2202
2,Тюрина Татьяна Игоревна,2014_Тюрина_Татьяна_Игоревна,2014,2.2,2,2,209,«Уже во второй раз избираюсь в студенческий со...,https://vk.com/wall-33327655_2202
3,Дорман Елена Вячеславовна,2014_Дорман_Елена_Вячеславовна,2014,2.3,2,3,226,"«По моему мнению, за прошедший год я не плохо ...",https://vk.com/wall-33327655_2203
4,Оськин Александр Андреевич,2014_Оськин_Александр_Андреевич,2014,2.3,2,3,225,«Прежде всего здравствуйте:)\nНа протяжении по...,https://vk.com/wall-33327655_2203
...,...,...,...,...,...,...,...,...,...
357,Майданик Анна Кирилловна,2021_Майданик_Анна_Кирилловна,2021,1.1,1,1,107,"Добрый день!\nЧто может быть интереснее, чем у...",https://vk.com/wall-33327655_9970
358,Кадников Климентий Сергеевич,2021_Кадников_Климентий_Сергеевич,2021,1.1,1,1,101,"Тэкс.. Значит студсовет, да.\nНу, увидев меня ...",https://vk.com/wall-33327655_9970
359,Мазитова Вероника Рустэмовна,2021_Мазитова_Вероника_Рустэмовна,2021,6.1,6,1,606,"Всем привет! Попав в студенческий совет, покин...",https://vk.com/wall-33327655_9978
360,Терехов Михаил Сергеевич,2021_Терехов_Михаил_Сергеевич,2021,6.1,6,1,602,За правду!,https://vk.com/wall-33327655_9978


In [161]:
ssmm_candidates_info.to_csv('ssmm_candidates_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)


 ### creating tables about results

In [ ]:
'''
БАГ:
- https://vk.com/wall-33327655_7695 - не считывается поток 
'''

In [145]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

result_streams = {}
for link in result_links.keys():
    result_streams[link] = {
        'year_x_year_stream': str(result_links[link]['link_year_no']) + '_' + result_links[link]['stream_election_info']['year_stream_no'],
        'link_date': result_links[link]['link_date'],
        'link_year_no': result_links[link]['link_year_no'], 
        'link_month_no': result_links[link]['link_month_no'], 
        'link_day_no': result_links[link]['link_day_no'], 
        'link_time': result_links[link]['link_time'], 
        'year_stream_no': result_links[link]['stream_election_info']['year_stream_no'], 
        'year_no': result_links[link]['stream_election_info']['year_no'], 
        'stream_no': result_links[link]['stream_election_info']['stream_no'], 
        'stream_size': result_links[link]['stream_election_info']['stream_size'], 
        'voted_cnt': result_links[link]['stream_election_info']['voted_cnt'], 
        'invalid_cnt': result_links[link]['stream_election_info']['invalid_cnt'], 
        'NOTA_cnt': result_links[link]['stream_election_info']['NOTA_cnt'], 
        'organizers': result_links[link]['stream_election_info']['organizers']
    }
# announcement_streams

result_streams_df =  pd.DataFrame(result_streams.items(), columns=['link', 'params'])
result_streams_df_2 = pd.concat([result_streams_df.drop(['params'], axis=1), result_streams_df['params'].apply(pd.Series)], axis=1)
result_streams_df_3 = result_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
result_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers
0,https://vk.com/wall-33327655_10001,2021_1.1,2021-10-17,2021,10,17,22:02,1.1,1,1,141,71,0,8,"[Георгием Малиновским, Дарьей Мельниковой]"
1,https://vk.com/wall-33327655_10009,2021_1.2,2021-10-19,2021,10,19,14:08,1.2,1,2,138,73,0,15,"[Георгием Малиновским, Дарьей Мельниковой]"
2,https://vk.com/wall-33327655_10000,2021_1.3,2021-10-17,2021,10,17,21:05,1.3,1,3,109,63,0,4,[Петром Третьяковым]
3,https://vk.com/wall-33327655_9998,2021_2.1,2021-10-17,2021,10,17,20:02,2.1,2,1,127,66,0,6,"[Дарьей Мельниковой, Георгием Малиновским]"
4,https://vk.com/wall-33327655_9985,2021_2.2,2021-10-16,2021,10,16,19:05,2.2,2,2,122,70,0,4,[Дарьей Мельниковой]
5,https://vk.com/wall-33327655_9989,2021_2.3,2021-10-16,2021,10,16,23:03,2.3,2,3,95,48,0,0,"[Дарьей Мельниковой, Петром Третьяковым]"
6,https://vk.com/wall-33327655_9984,2021_2.5,2021-10-16,2021,10,16,17:02,2.5,2,5,21,11,0,0,[Георгием Малиновским]
7,https://vk.com/wall-33327655_10007,2021_3.1,2021-10-19,2021,10,19,11:05,3.1,3,1,105,53,0,4,"[Георгием Малиновским, Кириллом Дехничем]"
8,https://vk.com/wall-33327655_9992,2021_3.2,2021-10-17,2021,10,17,15:35,3.2,3,2,104,52,0,4,"[Дарьей Мельниковой, Кириллом Дехничем]"
9,https://vk.com/wall-33327655_9976,2021_3.3,2021-10-15,2021,10,15,23:11,3.3,3,3,97,52,0,6,"[Дарьей Мельниковой, Георгием Малиновским]"


In [146]:
result_streams_df_3.to_csv('ssmm_results_info_conv11.csv', sep='\t', encoding='utf-8', index=False)

In [151]:
# full table for 4-11 convocations:

list_of_df = []
for i in range(4,11+1):
    df = pd.read_csv('ssmm_results_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
    list_of_df.append(df)

ssmm_results_info = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
ssmm_results_info


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers
0,https://vk.com/wall-33327655_2250,2014_1.1,2014-11-21,2014,11,21,22:15,1.1,1,1,166,129,10,5,"['Ильёй Денисовым', 'Анастасией Сурковой']"
1,https://vk.com/wall-33327655_2252,2014_1.2,2014-11-21,2014,11,21,22:25,1.2,1,2,136,95,4,6,"['Ираклием Глунчадзе', 'Анастасией Сурковой']"
2,https://vk.com/wall-33327655_2254,2014_1.3,2014-11-21,2014,11,21,22:35,1.3,1,3,138,83,1,2,"['Ираклием Глунчадзе', 'Анастасией Савченко']"
3,https://vk.com/wall-33327655_2329,2014_2.1,2014-11-27,2014,11,27,23:15,2.1,2,1,135,89,1,1,"['Ираклием Глунчадзе', 'Анастасией Сурковой']"
4,https://vk.com/wall-33327655_2215,2014_2.2,2014-11-17,2014,11,17,21:06,2.2,2,2,135,87,0,7,['Глунчадзе Ираклием']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,https://vk.com/wall-33327655_9912,2021_5.3,2021-10-12,2021,10,12,20:20,5.3,5,3,89,46,0,1,['Георгием Малиновским']
141,https://vk.com/wall-33327655_9878,2021_5.4,2021-10-06,2021,10,6,20:36,5.4,5,4,37,21,0,0,['Кириллом Дехничем']
142,https://vk.com/wall-33327655_10026,2021_6.1,2021-10-21,2021,10,21,15:44,6.1,6,1,87,44,0,6,"['Георгием Малиновским', 'Дарьей Мельниковой']"
143,https://vk.com/wall-33327655_10031,2021_6.3,2021-10-21,2021,10,21,22:02,6.3,6,3,93,47,0,3,"['Георгием Малиновским', 'Петром Третьяковым']"


In [152]:
ssmm_results_info.to_csv('ssmm_results_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)


In [ ]:
# СДЕЛАТЬ ОТДЕЛЬНУЮ ТАБЛИЦУ ДЛЯ ОРГАНИЗАТОРОВ И ПОВЕРНУТЬ ТАК, ЧТОБЫ КАЖДЫЙ ОРГ БЫЛ НА СВОЕЙ СТРОКЕ

In [147]:
organizers_df = result_streams_df_3[['year_x_year_stream','organizers']].apply(lambda x: x.explode()).reset_index().drop(['index'], axis=1)
organizers_df


,year_x_year_stream,organizers
0,2021_1.1,Георгием Малиновским
1,2021_1.1,Дарьей Мельниковой
2,2021_1.2,Георгием Малиновским
3,2021_1.2,Дарьей Мельниковой
4,2021_1.3,Петром Третьяковым
5,2021_2.1,Дарьей Мельниковой
6,2021_2.1,Георгием Малиновским
7,2021_2.2,Дарьей Мельниковой
8,2021_2.3,Дарьей Мельниковой
9,2021_2.3,Петром Третьяковым


In [148]:
organizers_df.to_csv('ssmm_organizers_info_conv11.csv', sep='\t', encoding='utf-8', index=False)

In [153]:
# full table for 4-11 convocations:

list_of_df = []
for i in range(4,11+1):
    df = pd.read_csv('ssmm_organizers_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
    list_of_df.append(df)

ssmm_organizers_info = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
ssmm_organizers_info


,year_x_year_stream,organizers
0,2014_1.1,Ильёй Денисовым
1,2014_1.1,Анастасией Сурковой
2,2014_1.2,Ираклием Глунчадзе
3,2014_1.2,Анастасией Сурковой
4,2014_1.3,Ираклием Глунчадзе
...,...,...
284,2021_6.1,Дарьей Мельниковой
285,2021_6.3,Георгием Малиновским
286,2021_6.3,Петром Третьяковым
287,2021_m1,Георгием Малиновским


In [154]:
ssmm_organizers_info.to_csv('ssmm_organizers_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)


In [162]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

result_names = {}
for link in result_links.keys():
    for name in result_links[link]['stream_election_info']['dict_nominated_with_votes'].keys():
        result_names[name] = {
            'link_year_no_x_name': str(result_links[link]['link_year_no']) + "_" + "_".join(name.split()),
            'link_year_no': result_links[link]['link_year_no'], 
            'year_stream_no': result_links[link]['stream_election_info']['year_stream_no'], 
            'year_no': result_links[link]['stream_election_info']['year_no'], 
            'stream_no': result_links[link]['stream_election_info']['stream_no'], 
            'votes_cnt': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['votes_cnt'],
            'elected_flg': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['elected_flg'],
            'link': link
        }

result_names_df =  pd.DataFrame(result_names.items(), columns=['name', 'params'])
result_names_df_2 = pd.concat([result_names_df.drop(['params'], axis=1), result_names_df['params'].apply(pd.Series)], axis=1)
result_names_df_2
# .head(10)


,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,votes_cnt,elected_flg,link
0,Суркова Анастасия Юрьевна,2014_Суркова_Анастасия_Юрьевна,2014,2.2,2,2,53,1,https://vk.com/wall-33327655_2215
1,Денисов Илья Олегович,2014_Денисов_Илья_Олегович,2014,2.2,2,2,61,1,https://vk.com/wall-33327655_2215
2,Тюрина Татьяна Игоревна,2014_Тюрина_Татьяна_Игоревна,2014,2.2,2,2,48,1,https://vk.com/wall-33327655_2215
3,Дорман Елена Вячеславовна,2014_Дорман_Елена_Вячеславовна,2014,2.3,2,3,71,1,https://vk.com/wall-33327655_2216
4,Оськин Александр Андреевич,2014_Оськин_Александр_Андреевич,2014,2.3,2,3,45,1,https://vk.com/wall-33327655_2216
5,Хапкин Артём Владимирович,2014_Хапкин_Артём_Владимирович,2014,4.1,4,1,32,1,https://vk.com/wall-33327655_2240
6,Тулякова Анна Владимировна,2014_Тулякова_Анна_Владимировна,2014,4.1,4,1,19,1,https://vk.com/wall-33327655_2240
7,Матвеев Дмитрий Александрович,2014_Матвеев_Дмитрий_Александрович,2014,4.1,4,1,45,1,https://vk.com/wall-33327655_2240
8,Бунеев Артём Валерьевич,2014_Бунеев_Артём_Валерьевич,2014,4.2,4,2,44,1,https://vk.com/wall-33327655_2241
9,Борисов Василий Александрович,2014_Борисов_Василий_Александрович,2014,4.2,4,2,49,1,https://vk.com/wall-33327655_2241


In [163]:
result_names_df_2.to_csv('ssmm_votes_info_conv4.csv', sep='\t', encoding='utf-8', index=False)

In [164]:
# full table for 4-11 convocations:

list_of_df = []
for i in range(4,11+1):
    df = pd.read_csv('ssmm_votes_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
    list_of_df.append(df)

ssmm_votes_info = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
ssmm_votes_info


,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,votes_cnt,elected_flg,link
0,Суркова Анастасия Юрьевна,2014_Суркова_Анастасия_Юрьевна,2014,2.2,2,2,53,1,https://vk.com/wall-33327655_2215
1,Денисов Илья Олегович,2014_Денисов_Илья_Олегович,2014,2.2,2,2,61,1,https://vk.com/wall-33327655_2215
2,Тюрина Татьяна Игоревна,2014_Тюрина_Татьяна_Игоревна,2014,2.2,2,2,48,1,https://vk.com/wall-33327655_2215
3,Дорман Елена Вячеславовна,2014_Дорман_Елена_Вячеславовна,2014,2.3,2,3,71,1,https://vk.com/wall-33327655_2216
4,Оськин Александр Андреевич,2014_Оськин_Александр_Андреевич,2014,2.3,2,3,45,1,https://vk.com/wall-33327655_2216
...,...,...,...,...,...,...,...,...,...
367,Маммагаджиев Руслан Гаджимагомедович,2021_Маммагаджиев_Руслан_Гаджимагомедович,2021,m1,1,0,10,1,https://vk.com/wall-33327655_10018
368,Третьякова Екатерина Алексеевна,2021_Третьякова_Екатерина_Алексеевна,2021,m1,1,0,19,1,https://vk.com/wall-33327655_10018
369,Мазитова Вероника Рустэмовна,2021_Мазитова_Вероника_Рустэмовна,2021,6.1,6,1,27,1,https://vk.com/wall-33327655_10026
370,Терехов Михаил Сергеевич,2021_Терехов_Михаил_Сергеевич,2021,6.1,6,1,16,1,https://vk.com/wall-33327655_10026


In [165]:
ssmm_votes_info.to_csv('ssmm_votes_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)


## Completeness testing

In [ ]:
# ПОПРИДЖОЙНИВАТЬ АНОНСЫ НА РЕЗУЛЬТАТЫ, ЧТОБЫ ПОНЯТЬ, КАКИЕ ПОСТЫ С РЕЗУЛЬТАТАМИ НЕ РАСПОЗНАЛИСЬ

# - по созывам, внутри созывов:
#     - по потокам
#     - по кандидатам

In [166]:
ssmm_announcements_info = pd.read_csv('ssmm_announcement_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_candidates_info = pd.read_csv('ssmm_candidates_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_results_info = pd.read_csv('ssmm_results_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_organizers_info = pd.read_csv('ssmm_organizers_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_votes_info = pd.read_csv('ssmm_votes_info_conv_4_11.csv', sep='\t', encoding='utf-8')


In [235]:
req_announcement = requests.get('https://vk.com/wall-33327655_2225')
html_doc = req_announcement.text
print(html_doc)

            <!DOCTYPE html>
      <html lang="ru" class="vk vk_js_no vk_1x vk_flex_no r d h  vk_appAuth_no n vk_old  vk_schemes_no   ">
      <head>
              <meta charset="utf-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, minimal-ui, user-scalable=no" />
        <meta name="format-detection" content="telephone=no" />
        <meta http-equiv="X-UA-Compatible" content="IE=edge" />
        <meta name="MobileOptimized" content="176" />
        <meta name="HandheldFriendly" content="True" />
        <base id="base">
        
        <meta name="description" content="#выборы@stud_mm" />
        <meta property="og:description" content="#выборы@stud_mm" />
        <link rel="shortcut icon" href="/images/icons/favicons/fav_logo.ico?8"></link>
            <meta name="theme-color" content="#ffffff" />
    <link rel="icon" type="image/png" sizes="32x32" href="/images/icons/pwa/favicon_32.png?12">
    <link rel="apple-

In [98]:
# КОГДА ДОПИСАЛИ КАНДИДАТОВ В КОММЕНТ (единичный случай, забить)

from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://vk.com/wall-33327655_2225"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

text_by_strings = [text for text in soup.stripped_strings]
print(text_by_strings)

# for line in text_by_strings:
    

['#выборы@stud_mm Насыщенная выборная.. | Студенческий совет мехмата МГУ | ВКонтакте', 'Запись на стене', 'Студенческий совет мехмата МГУ', '19 ноя 2014 в 21:07', '#выборы@stud_mm', 'Насыщенная выборная неделя продолжается! Завтра на очереди четыре разных курса. А начнём с прекрасного, ведь две представительницы прекрасного пола избираются в студсовет от потока 1.2. До 00:00 ещё можно стать кандидатом вместе с ними.', 'Кандидаты представлены в том порядке, в котором приходили их заявки. Орфография и пунктуация сохранены. Напомним, можно будет поставить подписи не более чем за трёх (в данном случае двух) кандидатов либо одну подпись «против всех».', '1 КУРС, 2 ПОТОК (выборы завтра на первой паре в аудитории П3)', '— НАГОРНЫХ Яна Валерьевна, группа 108', '«О себе: активная, жизнерадостная, общительная:) Насчет планов. По большей части планирую заниматься организацией и проведением праздников и всяческих мероприятий на мехмате. Здесь, как уже можно было заметить, достаточно веселая и ярка